In [1]:
!pip install langchain_groq langchain_core langchain_community

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile",
)
result = llm.invoke("who is lord jesus christ?")
print(result.content)

Lord Jesus Christ is the central figure of Christianity, and his life, teachings, death, and resurrection are the foundation of the Christian faith. He is considered the Son of God, the Messiah, and the Savior of humanity.

According to the Bible, Jesus was born of the Virgin Mary in Bethlehem, and his birth is seen as a fulfillment of Old Testament prophecies. He grew up in Nazareth, and at the age of 30, he began his public ministry, preaching the kingdom of God and performing miracles.

Jesus' teachings emphasized the importance of love, compassion, and forgiveness. He taught that God is a loving Father who desires a personal relationship with each person, and that salvation comes through faith in him. He also taught about the importance of living a moral life, loving one's neighbors, and caring for the poor and marginalized.

Jesus' ministry was marked by controversy, and he was eventually arrested, tried, and sentenced to death by crucifixion. However, his death was not the end, a

In [3]:
!pip install pypdf

In [4]:
!pip install chromadb

In [5]:
!pip install sentence_transformers

In [6]:
#create_vector_db()

In [7]:
!pip install gradio

In [8]:
!pip install -U langchain-community

In [9]:
!pip install langchain-groq


In [10]:
!pip install pypdf

In [11]:
!pip install chromadb

In [44]:
# ╔════════ OPTIONAL first-time installs (≈1–2 min) ════════╗
# !pip install -q gradio transformers sentencepiece pandas matplotlib pillow \
#                 langchain langchain-groq chromadb unstructured pdfminer.six pypdf
# ╚═════════════════════════════════════════════════════════╝


import os, re, time, io, base64, textwrap, warnings, pathlib, pandas as pd
from PIL import Image
import gradio as gr
from transformers               import pipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter    import RecursiveCharacterTextSplitter
from langchain.vectorstores     import Chroma
from langchain.embeddings       import HuggingFaceBgeEmbeddings
from langchain.memory           import ConversationBufferMemory
from langchain.chains           import ConversationalRetrievalChain
from langchain.prompts          import PromptTemplate
from langchain_groq             import ChatGroq
from groq                       import RateLimitError

warnings.filterwarnings("ignore")

# ────────────────────────── CONFIG ───────────────────────────
API_KEY   = "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"     # 🔑  PUT YOUR GROQ KEY
MODEL     = "llama-3.3-70b-versatile"
DB_PATH   = "./chromadb"
LOG_FILE  = "./chat_transcript.txt"
PFP_PATH  = "profile.jpg"                      # portrait (square works best)

memory  = ConversationBufferMemory(memory_key="chat_history",
                                   return_messages=True,
                                   output_key="answer")

summariser = pipeline("summarization", model="t5-small",
                      truncation=True, max_length=256)

asr = pipeline("automatic-speech-recognition",
               model="openai/whisper-tiny", device=-1)    # CPU friendly

vector_db, qa_chain = None, None
pdf_text            = ""
cooldown_until      = 0.0
ask_me_later_queue  = []                                   # queued Qs

# ───────────────────── GLOSSARY DICT ────────────────────────
GLOSSARY = {
    "kpi"   : "Key Performance Indicator — measurable performance metric.",
    "etl"   : "Extract, Transform, Load — consolidating data pipeline.",
    "olap"  : "Online Analytical Processing — fast multidimensional analysis.",
    "cohort": "Group of users sharing a characteristic over a period.",
    "churn" : "Rate at which customers stop using a product or service."
}

# ──────────────────── LangChain helpers ─────────────────────
def new_llm() -> ChatGroq:
    return ChatGroq(
        temperature = 0,
        groq_api_key = API_KEY,
        model_name   = MODEL,
        max_tokens   = 256
    )

def load_or_build_db(chunks=None):
    emb = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    if chunks:
        db = Chroma.from_documents(chunks, emb, persist_directory=DB_PATH)
        db.persist();  return db
    if os.path.exists(DB_PATH):
        return Chroma(persist_directory=DB_PATH, embedding_function=emb)
    return None

def make_chain(vdb):
    prompt = PromptTemplate(
        template = ("You are **GenAI Analyst Copilot**. Respond concisely.\n\n"
                    "{context}\nUser: {question}\nAssistant:"),
        input_variables=["context","question"]
    )
    return ConversationalRetrievalChain.from_llm(
        llm        = new_llm(),
        retriever  = vdb.as_retriever(search_kwargs={"k":3}),
        memory     = memory,
        combine_docs_chain_kwargs={"prompt":prompt},
        return_source_documents = False,
        output_key = "answer"
    )

vector_db = load_or_build_db()
qa_chain  = make_chain(vector_db) if vector_db else None

# ───────────────────── PDF UPLOAD ───────────────────────────
def handle_upload(files, progress=gr.Progress(track_tqdm=True)):
    global vector_db, qa_chain, pdf_text
    docs, summaries = [], []
    pdf_text = ""

    for f in progress.tqdm(files or [], desc="🔍 Parsing PDFs"):
        name  = os.path.basename(f.name)
        pages = PyPDFLoader(f.name).load()
        docs += pages
        raw   = " ".join(p.page_content for p in pages)
        pdf_text += raw + " "
        summ_chunk = raw[:1000]
        summ = summariser(summ_chunk, min_length=40, max_length=120,
                          truncation=True, do_sample=False)[0]["summary_text"]
        summaries.append(f"### **{name}**\n{textwrap.fill(summ, 90)}")

    if docs:
        chunks     = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50).split_documents(docs)
        vector_db  = load_or_build_db(chunks)
        qa_chain   = make_chain(vector_db)

    status  = f"✅ Processed {len(docs)} PDF(s)."
    summary = "\n\n---\n\n".join(summaries) or "No summaries."
    return status, summary

# ───────────────────── VOICE → CHAT ────────────────────────
def transcribe_and_chat(audio):
    if audio is None:
        return "No audio provided.", ""
    user_txt = asr(audio)["text"].strip()
    return f"**You said:** {user_txt}", chat_response(user_txt, "")

# ──────────────── QUIZ QUESTION GEN ────────────────────────
def gen_questions():
    if not pdf_text:
        return "⚠️ Upload at least one PDF first."
    prompt = ("Generate 5 short quiz questions (no answers) about this document:\n\n"
              f"{pdf_text[:4000]}")
    try:
        return new_llm().invoke(prompt).content
    except Exception:
        return "Generation unavailable (rate-limit?)."

# ─────────────────── GLOSSARY LOOKUP ───────────────────────
def define_term(term):
    term = term.strip()
    if not term:
        return "Enter a term."
    low = term.lower()
    if low in GLOSSARY:
        return f"**{term}**: {GLOSSARY[low]}"
    try:
        return new_llm().invoke(f"Explain the BI term '{term}' briefly.").content
    except Exception:
        return "Definition service unavailable."

# ───────────────────── CHAT CORE ───────────────────────────
def _parse_wait(msg):
    m = re.search(r"try again in (\d+)m([\d\.]+)s",msg) or re.search(r"try again in ([\d\.]+)s",msg)
    if not m: return 300
    return int(float(m.group(1))*60+float(m.group(2))) if len(m.groups())==2 else int(float(m.group(1)))

def chat_response(user, history):
    """Stream with typing indicator & ask-me-later queue"""
    global cooldown_until
    if time.time() < cooldown_until:
        ask_me_later_queue.append(user)
        yield "🕒 Added to queue — I'll answer soon."
        return

    if qa_chain is None:
        yield "📂 Upload a PDF first."
        return

    # typing indicator
    yield "💬 Bot is typing ..."
    try:
        ans = qa_chain({"question":user})["answer"]
        with open(LOG_FILE, "a") as f:
            f.write(f"User: {user}\nBot: {ans}\n\n")
        yield ans
    except RateLimitError as e:
        cooldown_until = time.time() + _parse_wait(str(e))
        ask_me_later_queue.append(user)
        yield "⚠️ Groq quota — queued your question."

# ─────────────── SERVE QUEUED Qs ──────────────────────────
def serve_queue():
    if time.time() < cooldown_until or not ask_me_later_queue or qa_chain is None:
        return ""
    q = ask_me_later_queue.pop(0)
    ans = qa_chain({"question":q})["answer"]
    return f"**Queued Q:** {q}\n\n{ans}"

# ───────────── TRANSCRIPT DOWNLOAD ────────────────────────
def get_transcript():
    if not os.path.exists(LOG_FILE):
        pathlib.Path(LOG_FILE).write_text("Chat started\n")
    return LOG_FILE

# ───────────── PROFILE PHOTO (circle) ─────────────────────
pfp64 = ""
if os.path.exists(PFP_PATH):
    pfp64 = base64.b64encode(open(PFP_PATH,"rb").read()).decode()
pfp_html = (f'<div class="pfp"><img src="data:image/jpeg;base64,{pfp64}"/>'
            '<div class="pfp-name">Akshara</div></div>') if pfp64 else ""

# ───────────── EXTRA CSS (Times New Roman) ────────────────
extra_css = f"""
<style>
@keyframes wave {{
  0%{{background-position:0 50%}}50%{{background-position:100% 50%}}100%{{background-position:0 50%}}
}}
@keyframes sparkle {{
  0%{{transform:translateY(0) scale(1);opacity:.8}}
  100%{{transform:translateY(-80vh) scale(.6);opacity:0}}
}}
body{{--c1:#fde4ec;--c2:#f9d0ee;--c3:#fceaf2;
  background:linear-gradient(135deg,var(--c1),var(--c2),var(--c3),var(--c1));
  background-size:400% 400%;animation:wave 24s ease infinite;
  font-family:'Times New Roman',serif;text-align:center;
}}
body.dark{{--c1:#e4c7ff;--c2:#f4c2d7;--c3:#eec8ff}}
*{{font-family:'Times New Roman',serif}}
h1{{font-size:37px;margin:18px 0 10px;font-weight:700}}
h3#agenda{{font-size:27px;margin:12px 0 18px;font-weight:400}}
.gr-chat-message{{border:1px solid #d0c0ff;border-radius:14px}}
.gr-chat-message.from-user{{background:#f7eafd}}
.gr-chat-message.from-bot{{background:#efeaff}}
input:focus,textarea:focus,button:focus{{outline:none;box-shadow:0 0 0 3px #d1c3ff,0 0 8px #eec6ff}}
.sparkle{{position:fixed;width:6px;height:6px;border-radius:50%;
         background:#fff;box-shadow:0 0 6px #fff;animation:sparkle 9s linear infinite;
         pointer-events:none}}
.pfp{{position:fixed;top:18px;right:26px;text-align:center;z-index:999}}
.pfp img{{width:64px;height:64px;border-radius:50%;object-fit:cover;
         border:2px solid #fff;box-shadow:0 0 6px rgba(0,0,0,.25)}}
.pfp-name{{font-size:14px;margin-top:4px;color:#000}}
#toggle-btn{{position:fixed;bottom:20px;right:20px;border:none;background:#fed9f4;
            font-size:22px;border-radius:50%;width:44px;height:44px;
            box-shadow:0 0 6px rgba(0,0,0,.25);cursor:pointer}}
</style>
"""

# add 25 floating sparkles at fixed positions
sparkles_html = "".join(f'<div class="sparkle" style="left:{i}%"></div>'
                        for i in range(2,100,4))

# ───────────── EXTRA JS (cursor, confetti, drag, queue) ────────────
js_code = """
// cursor shimmer trail
document.addEventListener('mousemove',e=>{
  const d=document.createElement('div');
  d.className='sparkle';
  d.style.left=(e.clientX-3)+'px';
  d.style.top =(e.clientY-3)+'px';
  d.style.opacity='1';
  document.body.appendChild(d);
  setTimeout(()=>d.remove(),700);
});

// trigger confetti on upload
window.addEventListener('message',e=>{
 if(e.data==='confetti'){
   import('https://cdn.jsdelivr.net/npm/canvas-confetti@1.6.0/dist/confetti.module.mjs')
     .then(m=>m.default({ origin:{ y:0.1 }, spread:90, particleCount:130 }));
 }
});

// draggable glossary
setTimeout(()=>{
  const head=document.getElementById('glossaryHeader');
  if(!head) return;
  let drag=0,offX,offY;
  head.style.cursor='move';
  head.onmousedown=e=>{drag=1;offX=e.clientX-head.parentElement.offsetLeft;
                        offY=e.clientY-head.parentElement.offsetTop};
  document.onmouseup  =()=>drag=0;
  document.onmousemove=e=>{
     if(drag){
       head.parentElement.style.left=(e.clientX-offX)+'px';
       head.parentElement.style.top =(e.clientY-offY)+'px';
     }
  };
},1200);

// serve ask-me-later queue every 6 s
setInterval(()=>{google.colab.kernel.invokeFunction('serve_queue',[],{});},6000);
"""

# ───────────── THEME ──────────────────────
theme = gr.themes.Soft(primary_hue="indigo", secondary_hue="purple").set(
    body_background_fill = "#fde4ec",
    body_text_color      = "#1f1f2e",
    button_primary_background_fill = "#a58bff",
    button_primary_text_color      = "#fff",
    border_color_primary = "#c9bbff"
)

# ────────────────────  UI  ────────────────────────────
with gr.Blocks(theme=theme) as demo:
    # inject CSS, sparkles, profile & dark-mode toggle
    gr.HTML(extra_css + sparkles_html + pfp_html +
            '<button id="toggle-btn" onclick="document.body.classList.toggle(`dark`);'
            'this.textContent=document.body.classList.contains(`dark`)?`☀️`:`🌙`;">🌙</button>')

    # greeting
    hr = time.localtime().tm_hour
    part = "morning" if 5<=hr<12 else "afternoon" if 12<=hr<18 else "evening"
    gr.Markdown(f"<h1>InsightBot with GenAI</h1>")
    gr.Markdown(f"*Good {part}, Akshara <3*")

    # upload area
    upload   = gr.File(label="📁 Upload PDF File(s)",
                       file_types=[".pdf"], file_count="multiple")
    status   = gr.Textbox(label="Status", interactive=False)
    summary  = gr.Markdown()
    upload.change(handle_upload, upload, [status, summary])\
          .then(None, None, None,
                js="()=>window.postMessage('confetti','*')")  # trigger confetti

    gr.Markdown('<h3 id="agenda">What’s on your agenda today ?</h3>')

    # Chat with typing indicator
    chat_ui = gr.ChatInterface(chat_response)

    # Glossary (draggable box)
    with gr.Group(elem_id="glossary"):
        gr.Markdown('<div id="glossaryHeader"><strong>🔍 Glossary Lookup</strong></div>')
        term_in  = gr.Textbox(label="Enter BI term")
        term_btn = gr.Button("Define Term 📖")
        term_out = gr.Markdown()
        term_btn.click(define_term, term_in, term_out)

    # Quiz questions
    gr.Markdown("---  \n**❓ Generate Quiz Questions**")
    q_btn, q_out = gr.Button("Generate Questions"), gr.Markdown()
    q_btn.click(gen_questions, None, q_out)

    # Voice chat
    gr.Markdown("---  \n**🎙️ Voice Chat**")
    audio   = gr.Audio(type="filepath", label="Record then release")
    spoken  = gr.Markdown()
    reply   = gr.Markdown()
    audio.change(transcribe_and_chat, audio, [spoken, reply])

    # transcript download
    gr.Markdown("---  \n**📄 Download Chat Transcript**")
    dl_btn, dl_file = gr.Button("Download transcript 📥"), gr.File(interactive=False)
    dl_btn.click(get_transcript, None, dl_file)

    gr.Markdown("---  \n<small>GenAI Copilot ChatBot – made by **Akshara&nbsp;Avinash&nbsp;Sarode**</small>")

    # hidden element for queued answers
    queue_out = gr.Markdown(visible=False)
    demo.load(serve_queue, None, queue_out)
    demo.load(None, None, None, js=js_code)

# 🚀 launch
demo.launch(share=True)


Device set to use cpu
Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9ef4f66eea99cca4e3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
